# Image Processing Fundamentals

## Learning Objectives

By the end of this notebook, you will be able to:
- Convert between different color spaces (RGB, HSV, Grayscale)
- Track objects by color using HSV
- Apply various thresholding techniques
- Smooth and filter images to reduce noise
- Enhance image contrast with histogram equalization
- Denoise images effectively

---

---

**⏱️ Estimated Time**: 90-120 minutes  
**📚 Level**: Intermediate  
**📋 Prerequisites**: Completed notebooks 00-02

---

## Setup

Import required libraries:

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Configure matplotlib
%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 8)

print("Libraries imported successfully!")
print(f"OpenCV version: {cv.__version__}")

---

## Part 1: Color Space Conversions

### What are Color Spaces?

A **color space** is a specific organization of colors. Different color spaces are useful for different tasks:

- **BGR/RGB**: Standard color representation (Blue-Green-Red / Red-Green-Blue)
- **Grayscale**: Single channel (0-255), no color information
- **HSV**: Hue-Saturation-Value (best for color-based detection)
- **LAB**: Lightness + A/B color channels (perceptually uniform)

### Why HSV?

HSV is excellent for color tracking because:
- **Hue** represents the actual color (0-179 in OpenCV)
- **Saturation** represents color intensity (0-255)
- **Value** represents brightness (0-255)
- Separates color from brightness!

### Converting Color Spaces

In [ ]:
# Create a colorful test image
test_img = np.zeros((300, 500, 3), dtype=np.uint8)
test_img[0:100, :] = [0, 0, 255]  # Red top
test_img[100:200, :] = [0, 255, 0]  # Green middle
test_img[200:300, :] = [255, 0, 0]  # Blue bottom

# Convert to different color spaces
gray = cv.cvtColor(test_img, cv.COLOR_BGR2GRAY)
hsv = cv.cvtColor(test_img, cv.COLOR_BGR2HSV)
lab = cv.cvtColor(test_img, cv.COLOR_BGR2LAB)

# Display
plt.figure(figsize=(16, 8))

plt.subplot(2, 3, 1)
plt.imshow(cv.cvtColor(test_img, cv.COLOR_BGR2RGB))
plt.title("Original (BGR)")
plt.axis("off")

plt.subplot(2, 3, 2)
plt.imshow(gray, cmap="gray")
plt.title("Grayscale")
plt.axis("off")

plt.subplot(2, 3, 3)
plt.imshow(hsv)
plt.title("HSV")
plt.axis("off")

# Split and show HSV channels
h, s, v = cv.split(hsv)

plt.subplot(2, 3, 4)
plt.imshow(h, cmap="hsv")
plt.title("H (Hue) - Color")
plt.axis("off")

plt.subplot(2, 3, 5)
plt.imshow(s, cmap="gray")
plt.title("S (Saturation) - Intensity")
plt.axis("off")

plt.subplot(2, 3, 6)
plt.imshow(v, cmap="gray")
plt.title("V (Value) - Brightness")
plt.axis("off")

plt.tight_layout()
plt.show()

print(f"Original shape: {test_img.shape}")
print(f"Grayscale shape: {gray.shape}")
print(f"HSV shape: {hsv.shape}")

### Color Tracking with HSV

Let's track a specific color (red) using HSV:

In [ ]:
# Create an image with colored objects
scene = np.ones((400, 600, 3), dtype=np.uint8) * 200

# Add colored circles
cv.circle(scene, (150, 150), 60, (0, 0, 255), -1)  # Red circle
cv.circle(scene, (300, 150), 60, (0, 255, 0), -1)  # Green circle
cv.circle(scene, (450, 150), 60, (255, 0, 0), -1)  # Blue circle
cv.circle(scene, (150, 300), 60, (0, 0, 200), -1)  # Dark red circle
cv.circle(scene, (450, 300), 60, (0, 128, 255), -1)  # Orange circle

# Convert to HSV
hsv_scene = cv.cvtColor(scene, cv.COLOR_BGR2HSV)

# Define range for red color in HSV
# Red wraps around in HSV (0-10 and 170-180)
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 100, 100])
upper_red2 = np.array([180, 255, 255])

# Create masks
mask1 = cv.inRange(hsv_scene, lower_red1, upper_red1)
mask2 = cv.inRange(hsv_scene, lower_red2, upper_red2)
red_mask = cv.bitwise_or(mask1, mask2)

# Apply mask
red_only = cv.bitwise_and(scene, scene, mask=red_mask)

# Find contours of red objects
contours, _ = cv.findContours(red_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes
result = scene.copy()
for contour in contours:
    x, y, w, h = cv.boundingRect(contour)
    cv.rectangle(result, (x, y), (x + w, y + h), (0, 255, 0), 3)
    # Draw circle
    (cx, cy), radius = cv.minEnclosingCircle(contour)
    cv.circle(result, (int(cx), int(cy)), int(radius), (255, 0, 255), 2)

# Display
plt.figure(figsize=(18, 5))

plt.subplot(1, 4, 1)
plt.imshow(cv.cvtColor(scene, cv.COLOR_BGR2RGB))
plt.title("Original Scene")
plt.axis("off")

plt.subplot(1, 4, 2)
plt.imshow(red_mask, cmap="gray")
plt.title("Red Mask (HSV filtering)")
plt.axis("off")

plt.subplot(1, 4, 3)
plt.imshow(cv.cvtColor(red_only, cv.COLOR_BGR2RGB))
plt.title("Red Objects Only")
plt.axis("off")

plt.subplot(1, 4, 4)
plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
plt.title(f"Detection ({len(contours)} red objects found)")
plt.axis("off")

plt.tight_layout()
plt.show()

print(f"Red objects detected: {len(contours)}")

---

## Part 2: Image Thresholding

### What is Thresholding?

**Thresholding** converts grayscale images to binary (black and white) images. It's used for:
- Separating objects from background
- Document scanning
- Feature extraction
- Preparing images for further processing

### Simple Thresholding

Apply a fixed threshold value to all pixels.

In [ ]:
# Create a test image with varying intensity
threshold_img = np.zeros((300, 500), dtype=np.uint8)
for i in range(5):
    intensity = 50 + i * 40
    threshold_img[:, i * 100 : (i + 1) * 100] = intensity

# Add some shapes
cv.circle(threshold_img, (100, 150), 50, 200, -1)
cv.rectangle(threshold_img, (250, 100), (350, 200), 100, -1)
cv.rectangle(threshold_img, (400, 120), (480, 180), 255, -1)

# Apply different threshold types
_, binary = cv.threshold(threshold_img, 127, 255, cv.THRESH_BINARY)
_, binary_inv = cv.threshold(threshold_img, 127, 255, cv.THRESH_BINARY_INV)
_, trunc = cv.threshold(threshold_img, 127, 255, cv.THRESH_TRUNC)
_, tozero = cv.threshold(threshold_img, 127, 255, cv.THRESH_TOZERO)
_, tozero_inv = cv.threshold(threshold_img, 127, 255, cv.THRESH_TOZERO_INV)

# Display
plt.figure(figsize=(18, 10))

plt.subplot(2, 3, 1)
plt.imshow(threshold_img, cmap="gray")
plt.title("Original")
plt.axis("off")

plt.subplot(2, 3, 2)
plt.imshow(binary, cmap="gray")
plt.title("THRESH_BINARY\n(>127: white, ≤127: black)")
plt.axis("off")

plt.subplot(2, 3, 3)
plt.imshow(binary_inv, cmap="gray")
plt.title("THRESH_BINARY_INV\n(>127: black, ≤127: white)")
plt.axis("off")

plt.subplot(2, 3, 4)
plt.imshow(trunc, cmap="gray")
plt.title("THRESH_TRUNC\n(>127: 127, ≤127: unchanged)")
plt.axis("off")

plt.subplot(2, 3, 5)
plt.imshow(tozero, cmap="gray")
plt.title("THRESH_TOZERO\n(>127: unchanged, ≤127: 0)")
plt.axis("off")

plt.subplot(2, 3, 6)
plt.imshow(tozero_inv, cmap="gray")
plt.title("THRESH_TOZERO_INV\n(>127: 0, ≤127: unchanged)")
plt.axis("off")

plt.tight_layout()
plt.show()

### Adaptive Thresholding

Calculates threshold for small regions, better for varying lighting:

In [ ]:
# Create image with gradient lighting
gradient_img = np.zeros((300, 500), dtype=np.uint8)
for i in range(300):
    for j in range(500):
        gradient_img[i, j] = int((i / 300) * 150 + 50)

# Add text-like patterns
cv.putText(gradient_img, "ADAPTIVE", (50, 100), cv.FONT_HERSHEY_SIMPLEX, 2, 255, 4)
cv.putText(gradient_img, "THRESHOLD", (50, 200), cv.FONT_HERSHEY_SIMPLEX, 2, 255, 4)

# Simple threshold (fails with gradient)
_, simple_thresh = cv.threshold(gradient_img, 127, 255, cv.THRESH_BINARY)

# Adaptive threshold (handles gradient)
adaptive_mean = cv.adaptiveThreshold(
    gradient_img, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 11, 2
)
adaptive_gaussian = cv.adaptiveThreshold(
    gradient_img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2
)

# Display
plt.figure(figsize=(18, 10))

plt.subplot(2, 2, 1)
plt.imshow(gradient_img, cmap="gray")
plt.title("Original (Gradient Lighting)")
plt.axis("off")

plt.subplot(2, 2, 2)
plt.imshow(simple_thresh, cmap="gray")
plt.title("Simple Threshold (FAILS!)")
plt.axis("off")

plt.subplot(2, 2, 3)
plt.imshow(adaptive_mean, cmap="gray")
plt.title("Adaptive (Mean)")
plt.axis("off")

plt.subplot(2, 2, 4)
plt.imshow(adaptive_gaussian, cmap="gray")
plt.title("Adaptive (Gaussian)")
plt.axis("off")

plt.tight_layout()
plt.show()

print("Adaptive thresholding handles varying illumination!")

### Otsu's Thresholding

Automatically finds optimal threshold value:

In [ ]:
# Create bimodal image (two distinct intensity regions)
bimodal = np.zeros((300, 500), dtype=np.uint8)
bimodal[:, :250] = 80  # Dark region
bimodal[:, 250:] = 180  # Bright region

# Add noise
noise = np.random.normal(0, 20, bimodal.shape).astype(np.int16)
bimodal = np.clip(bimodal.astype(np.int16) + noise, 0, 255).astype(np.uint8)

# Add shapes
cv.circle(bimodal, (125, 150), 50, 60, -1)
cv.rectangle(bimodal, (320, 100), (420, 200), 200, -1)

# Simple threshold (manual)
_, manual_thresh = cv.threshold(bimodal, 127, 255, cv.THRESH_BINARY)

# Otsu's threshold (automatic)
otsu_value, otsu_thresh = cv.threshold(bimodal, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

# Display
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(bimodal, cmap="gray")
plt.title("Original (Bimodal)")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(manual_thresh, cmap="gray")
plt.title("Manual Threshold (127)")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(otsu_thresh, cmap="gray")
plt.title(f"Otsu's Threshold ({otsu_value:.0f})")
plt.axis("off")

plt.tight_layout()
plt.show()

print(f"Otsu automatically found optimal threshold: {otsu_value:.1f}")

---

## Part 3: Image Smoothing and Filtering

### Why Smooth Images?

- Reduce noise
- Blur details
- Prepare for edge detection
- Remove artifacts

### Understanding Kernels

A **kernel** (or filter) is a small matrix used to apply effects. Common sizes: 3×3, 5×5, 7×7

In [ ]:
# Create noisy image
clean_img = np.ones((300, 300), dtype=np.uint8) * 128
cv.circle(clean_img, (150, 150), 80, 200, -1)
cv.rectangle(clean_img, (50, 50), (100, 100), 50, -1)

# Add Gaussian noise
noise = np.random.normal(0, 25, clean_img.shape).astype(np.int16)
noisy_img = np.clip(clean_img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

# Add salt and pepper noise
salt_pepper = noisy_img.copy()
salt = np.random.random(salt_pepper.shape) > 0.97
pepper = np.random.random(salt_pepper.shape) < 0.03
salt_pepper[salt] = 255
salt_pepper[pepper] = 0

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(clean_img, cmap="gray")
plt.title("Clean Image")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(noisy_img, cmap="gray")
plt.title("With Gaussian Noise")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(salt_pepper, cmap="gray")
plt.title("With Salt & Pepper Noise")
plt.axis("off")

plt.tight_layout()
plt.show()

### Averaging (Box Filter)

In [ ]:
# Apply averaging with different kernel sizes
avg_3x3 = cv.blur(noisy_img, (3, 3))
avg_5x5 = cv.blur(noisy_img, (5, 5))
avg_9x9 = cv.blur(noisy_img, (9, 9))

plt.figure(figsize=(18, 10))

plt.subplot(2, 2, 1)
plt.imshow(noisy_img, cmap="gray")
plt.title("Noisy Original")
plt.axis("off")

plt.subplot(2, 2, 2)
plt.imshow(avg_3x3, cmap="gray")
plt.title("Averaging 3×3")
plt.axis("off")

plt.subplot(2, 2, 3)
plt.imshow(avg_5x5, cmap="gray")
plt.title("Averaging 5×5")
plt.axis("off")

plt.subplot(2, 2, 4)
plt.imshow(avg_9x9, cmap="gray")
plt.title("Averaging 9×9 (more blur)")
plt.axis("off")

plt.tight_layout()
plt.show()

### Gaussian Blurring

Better than averaging - gives more weight to nearby pixels:

In [ ]:
# Apply Gaussian blur
gaussian_3x3 = cv.GaussianBlur(noisy_img, (3, 3), 0)
gaussian_5x5 = cv.GaussianBlur(noisy_img, (5, 5), 0)
gaussian_9x9 = cv.GaussianBlur(noisy_img, (9, 9), 0)

plt.figure(figsize=(18, 10))

plt.subplot(2, 2, 1)
plt.imshow(noisy_img, cmap="gray")
plt.title("Noisy Original")
plt.axis("off")

plt.subplot(2, 2, 2)
plt.imshow(gaussian_3x3, cmap="gray")
plt.title("Gaussian Blur 3×3")
plt.axis("off")

plt.subplot(2, 2, 3)
plt.imshow(gaussian_5x5, cmap="gray")
plt.title("Gaussian Blur 5×5")
plt.axis("off")

plt.subplot(2, 2, 4)
plt.imshow(gaussian_9x9, cmap="gray")
plt.title("Gaussian Blur 9×9")
plt.axis("off")

plt.tight_layout()
plt.show()

### Median Filtering

Excellent for salt-and-pepper noise, preserves edges:

In [ ]:
# Compare on salt-and-pepper noise
avg_sp = cv.blur(salt_pepper, (5, 5))
gaussian_sp = cv.GaussianBlur(salt_pepper, (5, 5), 0)
median_sp = cv.medianBlur(salt_pepper, 5)

plt.figure(figsize=(18, 10))

plt.subplot(2, 2, 1)
plt.imshow(salt_pepper, cmap="gray")
plt.title("Salt & Pepper Noise")
plt.axis("off")

plt.subplot(2, 2, 2)
plt.imshow(avg_sp, cmap="gray")
plt.title("Averaging (Poor)")
plt.axis("off")

plt.subplot(2, 2, 3)
plt.imshow(gaussian_sp, cmap="gray")
plt.title("Gaussian (Poor)")
plt.axis("off")

plt.subplot(2, 2, 4)
plt.imshow(median_sp, cmap="gray")
plt.title("Median (Excellent!) ✓")
plt.axis("off")

plt.tight_layout()
plt.show()

print("Median filter is best for salt-and-pepper noise!")

### Bilateral Filtering

Reduces noise while preserving edges:

In [ ]:
# Create image with edges
edge_img = np.ones((300, 300), dtype=np.uint8) * 50
cv.rectangle(edge_img, (50, 50), (250, 250), 200, -1)
cv.circle(edge_img, (150, 150), 60, 100, -1)

# Add noise
noise = np.random.normal(0, 20, edge_img.shape).astype(np.int16)
noisy_edge = np.clip(edge_img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

# Compare filters
gaussian_edge = cv.GaussianBlur(noisy_edge, (9, 9), 0)
bilateral_edge = cv.bilateralFilter(noisy_edge, 9, 75, 75)

plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
plt.imshow(noisy_edge, cmap="gray")
plt.title("Noisy (with edges)")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(gaussian_edge, cmap="gray")
plt.title("Gaussian (blurs edges)")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(bilateral_edge, cmap="gray")
plt.title("Bilateral (preserves edges!) ✓")
plt.axis("off")

plt.tight_layout()
plt.show()

print("Bilateral filter preserves edges while smoothing!")

---

## Part 4: Histogram Equalization

### What is Histogram Equalization?

Improves image contrast by spreading out intensity values:

### 💡 Parameter Tuning Tips for CLAHE

**clipLimit** (default: 2.0):
- Lower values (1.0-2.0): Less contrast enhancement, more natural
- Higher values (3.0-8.0): More contrast, but may introduce noise
- Too high (>10): Over-enhancement, unnatural appearance

**tileGridSize** (default: (8,8)):
- Smaller tiles (4,4): More local adaptation, may create visible grid artifacts
- Larger tiles (16,16): Smoother results, less local adaptation
- Try: (8,8) for balanced results

**When to use**: Low-contrast images, medical images, underwater images

---

## Part 5: Image Denoising

### Non-Local Means Denoising

Advanced denoising technique that compares patches:

In [ ]:
# Create color image with noise
color_clean = np.zeros((200, 300, 3), dtype=np.uint8)
color_clean[:, :100] = [200, 100, 50]
color_clean[:, 100:200] = [50, 200, 100]
color_clean[:, 200:] = [100, 50, 200]

# Add significant noise
noise_color = np.random.normal(0, 30, color_clean.shape).astype(np.int16)
color_noisy = np.clip(color_clean.astype(np.int16) + noise_color, 0, 255).astype(np.uint8)

# Denoise
denoised = cv.fastNlMeansDenoisingColored(color_noisy, None, 10, 10, 7, 21)

# Display
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(cv.cvtColor(color_clean, cv.COLOR_BGR2RGB))
plt.title("Clean Original")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(cv.cvtColor(color_noisy, cv.COLOR_BGR2RGB))
plt.title("Noisy")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(cv.cvtColor(denoised, cv.COLOR_BGR2RGB))
plt.title("Denoised (NL-Means)")
plt.axis("off")

plt.tight_layout()
plt.show()

print("Non-Local Means denoising effectively removes noise!")

---

## Part 6: Practical Exercises

### Exercise 1: Build a Color Tracker

In [ ]:
# TODO: Create a scene and track blue objects
# Hint: Use HSV color space
# Blue in HSV: H=90-130, S=50-255, V=50-255

# Your code here!
exercise_scene = np.ones((400, 600, 3), dtype=np.uint8) * 180
cv.circle(exercise_scene, (100, 200), 50, (255, 0, 0), -1)  # Blue
cv.circle(exercise_scene, (300, 200), 50, (0, 255, 0), -1)  # Green
cv.circle(exercise_scene, (500, 200), 50, (200, 100, 0), -1)  # Teal (partial blue)

# Convert and detect blue
hsv_ex = cv.cvtColor(exercise_scene, cv.COLOR_BGR2HSV)
blue_mask = cv.inRange(hsv_ex, np.array([90, 50, 50]), np.array([130, 255, 255]))
blue_result = cv.bitwise_and(exercise_scene, exercise_scene, mask=blue_mask)

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(cv.cvtColor(exercise_scene, cv.COLOR_BGR2RGB))
plt.title("Scene")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(blue_mask, cmap="gray")
plt.title("Blue Mask")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(cv.cvtColor(blue_result, cv.COLOR_BGR2RGB))
plt.title("Blue Objects")
plt.axis("off")

plt.tight_layout()
plt.show()

### Exercise 2: Compare Thresholding Methods

In [ ]:
# TODO: Create image and compare simple, adaptive, and Otsu thresholding
# Which works best for which scenario?

print("Try creating different types of images (uniform lighting, gradient, etc.)")
print("and see which thresholding method works best!")

### Exercise 3: Denoise Comparison

In [ ]:
# TODO: Add different types of noise and test different filters
# - Gaussian noise → Which filter works best?
# - Salt & pepper → Which filter works best?
# - Mixed noise → Which filter works best?

print("Experiment with different noise types and filters!")

---

## Summary

Congratulations! You've completed Image Processing Fundamentals. You now know:

✓ Color space conversions (BGR, HSV, Grayscale)  
✓ Color-based object tracking with HSV  
✓ Thresholding techniques (simple, adaptive, Otsu's)  
✓ Image smoothing (averaging, Gaussian, median, bilateral)  
✓ Histogram equalization for contrast enhancement  
✓ Advanced denoising with NL-Means  

### Key Takeaways

1. **HSV is best for color detection** - separates color from brightness
2. **Adaptive thresholding handles varying light** - better than simple threshold
3. **Otsu finds optimal threshold automatically** - for bimodal images
4. **Median filter best for salt-and-pepper** - preserves edges
5. **Bilateral filter preserves edges** - while smoothing
6. **CLAHE better than histogram equalization** - avoids over-amplification

---

## What's Next?

In the next notebook (**04_image_transformations.ipynb**), you'll learn:
- Geometric transformations (scaling, rotation, translation)
- Perspective transformation
- Image warping
- Watermarking techniques

---

## Real-World Applications

- **Document Scanning**: Thresholding for clean text
- **Object Tracking**: HSV-based color detection
- **Medical Imaging**: Denoising and contrast enhancement
- **Quality Control**: Defect detection with thresholding
- **Photography**: Noise reduction and enhancement

---

**Happy Coding!** 🎨📊